# Training Notebook
---

## Load the Data
Use one of the cells below to either parse the data from the file structure or load the data from the saved numpy arrays

In [1]:
import utils
import numpy as np

image_files, labels = utils.getprocessedfilelist()
np.save(os.path.join('data_road', 'images.npy'), image_files)
np.save(os.path.join('data_road', 'labels.npy'), labels)

In [1]:
import numpy as np

# use this cell to load the numpy data from disk so that you
# don't always have to parse the file structure
image_files = np.load(os.path.join('data_road', 'images.npy'))
labels = np.load(os.path.join('data_road', 'labels.npy'))

## Create the Input Pipeline
---
Create an input pipeline from the loaded arrays. Split the pipeline into train and val datasets.

In [2]:
import tensorflow as tf

data = tf.data.Dataset.from_tensor_slices((image_files, labels))

@tf.function
def readimage(image_file, label):
    image = tf.io.decode_jpeg(tf.io.read_file(image_file)) / 255
    label = (label / 80) - 1
    return image, label

batch_size = 32
shuffle_buffer_size = len(labels)
train_ds_size = 20000
val_ds_size = 5000

shuffled_ds = data.shuffle(shuffle_buffer_size).map(readimage, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = shuffled_ds.take(train_ds_size).batch(batch_size)
val_ds = shuffled_ds.skip(train_ds_size).take(val_ds_size).batch(batch_size)

iter_ = shuffled_ds.as_numpy_iterator()
a = iter_.next()
print(a)

(array([[[0.16470589, 0.26666668, 0.16078432],
        [0.16470589, 0.2509804 , 0.16078432],
        [0.18431373, 0.22745098, 0.16078432],
        ...,
        [0.16078432, 0.23921569, 0.27450982],
        [0.14901961, 0.22745098, 0.25490198],
        [0.14509805, 0.22352941, 0.2509804 ]],

       [[0.15294118, 0.23137255, 0.13333334],
        [0.16470589, 0.23529412, 0.14901961],
        [0.19215687, 0.22745098, 0.16078432],
        ...,
        [0.14117648, 0.21960784, 0.25490198],
        [0.12941177, 0.20784314, 0.24313726],
        [0.12941177, 0.20784314, 0.23529412]],

       [[0.09803922, 0.13333334, 0.05882353],
        [0.1254902 , 0.15294118, 0.08235294],
        [0.16078432, 0.1764706 , 0.11764706],
        ...,
        [0.12156863, 0.19215687, 0.23921569],
        [0.12941177, 0.2       , 0.24705882],
        [0.14901961, 0.21960784, 0.25882354]],

       ...,

       [[0.23529412, 0.3254902 , 0.16078432],
        [0.23921569, 0.3372549 , 0.17254902],
        [0.25882354, 

## Create the Model
---

Create the model and add callbacks for writing checkpoints and logs

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, LSTM, MaxPool2D, Flatten, InputLayer, TimeDistributed
import tensorflow as tf
import os

def create_model():
    model = Sequential([
        InputLayer(input_shape=(160,600,3)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (5,1), strides=(4,1), activation='relu'),
        Conv2D(1, (4,1), activation='relu'),
        Flatten(),
        Dense(600, activation='tanh')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

model = create_model()


## Train!

In [4]:
tf.keras.backend.clear_session()

callbacks = [
    tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt','training-3-cnn'), 
        save_weights_only=True)
]

# model.load_weights(os.path.join('ckpt', 'training-1'))
model.fit(x=train_ds, batch_size=batch_size, epochs=80, callbacks=callbacks, validation_data=val_ds, verbose=1)

Epoch 1/80
290/625 [============>.................] - ETA: 1:10 - loss: 2866.3938 - accuracy: 0.0040

KeyboardInterrupt: 